to solve this question https://stackoverflow.com/questions/24612626/b-spline-interpolation-with-python/24693358#24693358

In [1]:
import os

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [3]:
import numpy as np
import scipy.interpolate as si

b spline interpolation from https://stackoverflow.com/questions/24612626/b-spline-interpolation-with-python

In [4]:
#curve base
points = [[-6, 2], [-3, -2],[0, 0], [0, 2], [2, 3], [4, 0], [6, 3], [8, 5], [8, 8], [6, 8], [5, 9], [7, 2]];
points = np.array(points)
x = points[:,0]
y = points[:,1]


t = range(len(points))
ipl_t = np.linspace(0.0, len(points) - 1, 100)


x_tup = si.splrep(t, x, k=3)
y_tup = si.splrep(t, y, k=3)

x_list = list(x_tup)
xl = x.tolist()
x_list[1] = xl + [0.0, 0.0, 0.0, 0.0]

y_list = list(y_tup)
yl = y.tolist()
y_list[1] = yl + [0.0, 0.0, 0.0, 0.0]

x_i = si.splev(ipl_t, x_list)
y_i = si.splev(ipl_t, y_list)

In [7]:
url = "https://cgm.demo-de.cloud.c3.cgm.ag/cgmg3"
service = Service(executable_path='./chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)

In [21]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time

driver.execute_script("""
    document.body.style.cursor = 'pointer';
""")

# Füge diesen JavaScript-Code in dein Selenium-Skript ein, bevor die Aktionen ausgeführt werden:
driver.execute_script("""
    var customCursor = document.getElementById('custom-cursor');
    if (customCursor) {
        customCursor.remove();
    }
    document.body.style.cursor = 'default';
""")



from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time

action = ActionChains(driver)

# Funktion zum langsamen Bewegen und Klicken
def move_and_click(element):
    action.move_to_element(element).perform()
    time.sleep(0.5)  # Langsame Bewegung vor dem Klick
    action.click().perform()
    time.sleep(0.5)

# Zuerst auf "Menü" klicken, das ein <a>-Element ist
menuElement = driver.find_element(By.XPATH, "//a[contains(text(), 'Menü')]")
move_and_click(menuElement)

# Menü erneut anklicken, um es wieder einzuklappen
move_and_click(menuElement)

# Dann auf "Fallsuche" klicken
fallsucheElement = driver.find_element(By.XPATH, "//label[contains(text(), 'Fallsuche')]")
move_and_click(fallsucheElement)

# Zuletzt auf "Übersicht Arzt" klicken
uebersichtArztElement = driver.find_element(By.XPATH, "//label[contains(text(), 'Übersicht Arzt')]")
move_and_click(uebersichtArztElement)
